In [1]:
import nltk
import spacy
import unicodedata
import re
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
import collections
from nltk.tokenize.toktok import ToktokTokenizer
import en_core_web_sm
import time

nltk.download('stopwords')

C:\Users\colto\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\colto\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\colto\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\colto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import os

In [3]:
from glob import iglob
import pandas as pd

path = r'C:\\Users\\colto\\OneDrive\\Desktop\\desktop stuff\\audio_files\**\*.csv'

all_rec = iglob(path, recursive=True)

# Extract only c3 column from files
dataframes = {f: pd.read_csv(f, usecols=['comment']) for f in all_rec}

# Group all dataframes then combine text rows of each dataframe
raw_df = pd.concat(dataframes).groupby(level=0)['comment'] \
                  .apply(lambda x: ' '.join(x.tolist())).reset_index(drop=True)

In [17]:
raw_df2=raw_df.to_frame()

In [18]:
raw_df2

,comment
0,mostly just going to go into a word document s...
1,go ahead. So how long have you been here? What...
2,I started recording um basically I think one o...
3,recording and then I'll just generally ask you...
4,she's so impersonal. Do you have any questions...
...,...
69,I started recording um basically I think one o...
70,mostly just going to go into a word document s...
71,mostly just going to go into a word document s...
72,mostly just going to go into a word document s...


In [20]:
raw_df3 = raw_df2.drop([71, 72, 73])

In [21]:
raw_df3

,comment
0,mostly just going to go into a word document s...
1,go ahead. So how long have you been here? What...
2,I started recording um basically I think one o...
3,recording and then I'll just generally ask you...
4,she's so impersonal. Do you have any questions...
...,...
66,sorry. No perfect person only because I can wr...
67,"Okay, so I guess the first question I really w..."
68,"Okay, so in general, if you were to describe y..."
69,I started recording um basically I think one o...


In [22]:
raw_df3.shape

(71, 1)

In [23]:
data_selection = raw_df3

In [24]:
data_selection.tail(5)

,comment
66,sorry. No perfect person only because I can wr...
67,"Okay, so I guess the first question I really w..."
68,"Okay, so in general, if you were to describe y..."
69,I started recording um basically I think one o...
70,mostly just going to go into a word document s...


In [25]:
# Remove punctuation
data_selection['survey_text_processed'] = data_selection['comment'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
data_selection['survey_text_processed'] = data_selection['survey_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
data_selection['survey_text_processed'].head()

0    mostly just going to go into a word document s...
1    go ahead so how long have you been here what d...
2    i started recording um basically i think one o...
3    recording and then i'll just generally ask you...
4    she's so impersonal do you have any questions ...
Name: survey_text_processed, dtype: object

In [43]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = data_selection.survey_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[1:30])

[['go', 'ahead', 'so', 'how', 'long', 'have', 'you', 'been', 'here', 'what', 'do', 'you', 'do', 'um', 'tech', 'ops', 'line', 'maintenance', 'years', 'so', 'anything', 'you', 'have', 'as', 'far', 'as', 'the', 'premiums', 'or', 'anything', 'like', 'that', 'goes', 've', 'been', 'there', 've', 'been', 'through', 'the', 'pay', 'cuts', 've', 'been', 'through', 'the', 'what', 'have', 'we', 'been', 'through', 'pay', 'cuts', 'different', 'about', 'different', 'um', 'couple', 'of', 'different', 'health', 'corporations', 'um', 'the', 'best', 'of', 'everything', 'and', 'then', 'the', 'worst', 'of', 'everything', 'we', 've', 'been', 'through', 'bankruptcy', 'and', 'so', 've', 'kind', 'of', 'been', 'here', 'long', 'enough', 'to', 'where', 'just', 'just', 'go', 'with', 'the', 'flow', 'nothing', 'nothing', 'excites', 'me', 'anymore', 'as', 'far', 'as', 'changes', 'um', 'when', 'we', 'did', 'not', 'get', 'our', 'profit', 'sharing', 'that', 'the', 'way', 'it', 'is', 'you', 'know', 'you', 'just', 'look',

In [44]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=35) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=35)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [45]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\colto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [47]:
import spacy
nlp.max_length = 1030000 # or higher
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp.max_length = 1300000 # or higher

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1:30])

[['go', 'ahead', 'long', 'tech_op', 'year', 'far', 'premium', 'go', 'different', 'different', 'couple', 'different', 'health', 'corporation', 'good', 'worst', 'bankruptcy', 'kind', 'long', 'enough', 'go', 'flow', 'excite', 'far', 'change', 'profit_share', 'way', 'know', 'look', 'other', 'expect', 'year', 'disappoint', 'know', 'say', 'lucky', 'job', 'mean', 'wrong', 'business', 'want', 'continue', 'make', 'lot', 'money', 'fact', 'think', 'say', 'want', 'make', 'want', 'make', 'dollar', 'airline', 'forget', 'say', 'big', 'big', 'guy', 'exactly', 'ride', 'go', 'go', 'college', 'start', 'cargo', 'finally', 'go', 'go', 'long', 'run', 'know', 'follow', 'money', 'tell', 'follow', 'money', 'well', 'follow', 'keep', 'say', 'want', 'know', 'want', 'mechanic', 'know', 'guy', 'would', 'work', 'car', 'neighborhood', 'know', 'part', 'would', 'work', 'car', 'work', 'bike', 'work', 'stuff', 'turn', 'turn', 'wrench', 'guess', 'year', 'still', 'reason', 'guess', 'kid', 'college', 'thing', 'kind', 'keep'

In [48]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[1:30])

[[(1, 1), (3, 1), (4, 3), (6, 1), (7, 6), (8, 1), (15, 5), (16, 1), (17, 1), (18, 18), (19, 1), (20, 3), (26, 8), (28, 2), (29, 1), (30, 2), (32, 13), (33, 1), (34, 10), (35, 8), (39, 1), (43, 28), (44, 8), (52, 7), (54, 11), (55, 2), (56, 1), (58, 2), (59, 8), (60, 3), (61, 1), (64, 1), (65, 2), (67, 10), (68, 6), (75, 9), (77, 2), (78, 1), (79, 4), (82, 1), (83, 11), (84, 3), (87, 2), (89, 2), (91, 6), (94, 1), (95, 2), (98, 3), (103, 4), (104, 2), (105, 1), (106, 2), (108, 51), (110, 4), (111, 11), (112, 88), (115, 10), (116, 6), (117, 4), (118, 3), (119, 61), (120, 8), (121, 1), (122, 1), (123, 3), (124, 4), (127, 9), (128, 2), (129, 1), (130, 1), (131, 7), (134, 1), (138, 2), (140, 4), (142, 6), (143, 144), (145, 3), (146, 4), (147, 5), (148, 2), (149, 1), (151, 1), (152, 2), (154, 9), (156, 1), (157, 1), (159, 6), (161, 11), (162, 4), (163, 7), (165, 10), (166, 16), (167, 9), (168, 3), (169, 11), (170, 1), (171, 8), (172, 8), (173, 21), (177, 1), (180, 4), (181, 10), (182, 5), (1

In [49]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=20,
                                       per_word_topics=True)

In [50]:
lda_model.print_topics()

[(0,
  '0.001*"know" + 0.000*"go" + 0.000*"think" + 0.000*"get" + 0.000*"people" + 0.000*"would" + 0.000*"time" + 0.000*"say" + 0.000*"want" + 0.000*"work"'),
 (1,
  '0.001*"go" + 0.001*"know" + 0.000*"think" + 0.000*"get" + 0.000*"say" + 0.000*"people" + 0.000*"would" + 0.000*"want" + 0.000*"thing" + 0.000*"work"'),
 (2,
  '0.051*"know" + 0.032*"go" + 0.016*"get" + 0.016*"think" + 0.014*"say" + 0.014*"people" + 0.013*"work" + 0.012*"time" + 0.012*"would" + 0.011*"thing"'),
 (3,
  '0.027*"know" + 0.025*"people" + 0.020*"say" + 0.019*"go" + 0.017*"think" + 0.012*"could" + 0.011*"come" + 0.010*"use" + 0.010*"feel" + 0.009*"get"'),
 (4,
  '0.001*"know" + 0.000*"go" + 0.000*"say" + 0.000*"people" + 0.000*"think" + 0.000*"get" + 0.000*"would" + 0.000*"want" + 0.000*"work" + 0.000*"day"'),
 (5,
  '0.001*"know" + 0.001*"go" + 0.000*"people" + 0.000*"would" + 0.000*"get" + 0.000*"say" + 0.000*"time" + 0.000*"work" + 0.000*"really" + 0.000*"want"'),
 (6,
  '0.072*"know" + 0.025*"go" + 0.019*"th

In [39]:
from pprint import pprint

# Print the Keyword in the 15 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"go" + 0.007*"know" + 0.003*"get" + 0.003*"think" + 0.003*"would" + '
  '0.002*"work" + 0.002*"say" + 0.002*"want" + 0.002*"come" + 0.002*"lot"'),
 (1,
  '0.010*"know" + 0.007*"go" + 0.003*"people" + 0.003*"think" + 0.003*"say" + '
  '0.003*"get" + 0.002*"thing" + 0.002*"time" + 0.002*"work" + 0.002*"want"'),
 (2,
  '0.003*"know" + 0.003*"go" + 0.002*"get" + 0.001*"say" + 0.001*"think" + '
  '0.001*"people" + 0.001*"work" + 0.001*"take" + 0.001*"time" + 0.001*"want"'),
 (3,
  '0.009*"know" + 0.004*"go" + 0.003*"get" + 0.003*"people" + 0.003*"work" + '
  '0.002*"would" + 0.002*"time" + 0.002*"think" + 0.002*"say" + '
  '0.002*"health"'),
 (4,
  '0.011*"know" + 0.007*"go" + 0.004*"think" + 0.003*"people" + 0.003*"say" + '
  '0.003*"work" + 0.003*"time" + 0.003*"get" + 0.002*"would" + 0.002*"day"'),
 (5,
  '0.005*"know" + 0.003*"go" + 0.003*"think" + 0.002*"people" + 0.002*"time" + '
  '0.002*"work" + 0.002*"say" + 0.002*"get" + 0.002*"flight" + 0.002*"really"'),
 (6,
  '0.0

In [51]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.21019412220718514


In [ ]:
#score is a bit low, might need to revise this.

In [52]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=41,
                                           chunksize=100,
                                           passes=30,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [54]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 10
max_topics = 20
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 600/600 [7:47:54<00:00, 46.79s/it]  


In [56]:
num_topics = 12

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=41,
                                           chunksize=100,
                                           passes=30,
                                           alpha=0.91,
                                           eta=0.91)

In [57]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"know" + 0.000*"go" + 0.000*"think" + 0.000*"get" + 0.000*"people" + '
  '0.000*"would" + 0.000*"time" + 0.000*"gradation" + 0.000*"work" + '
  '0.000*"inflammation"'),
 (1,
  '0.000*"know" + 0.000*"go" + 0.000*"think" + 0.000*"get" + 0.000*"thing" + '
  '0.000*"say" + 0.000*"lot" + 0.000*"people" + 0.000*"time" + 0.000*"health"'),
 (2,
  '0.000*"know" + 0.000*"go" + 0.000*"time" + 0.000*"think" + 0.000*"get" + '
  '0.000*"people" + 0.000*"gradation" + 0.000*"inflammation" + 0.000*"want" + '
  '0.000*"would"'),
 (3,
  '0.000*"know" + 0.000*"go" + 0.000*"gradation" + 0.000*"inflammation" + '
  '0.000*"people" + 0.000*"think" + 0.000*"get" + 0.000*"infection" + '
  '0.000*"coincidence" + 0.000*"unfair"'),
 (4,
  '0.000*"know" + 0.000*"go" + 0.000*"think" + 0.000*"people" + 0.000*"work" + '
  '0.000*"get" + 0.000*"time" + 0.000*"come" + 0.000*"say" + 0.000*"thing"'),
 (5,
  '0.000*"know" + 0.000*"go" + 0.000*"think" + 0.000*"would" + 0.000*"say" + '
  '0.000*"work" + 0.000*"

In [60]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(r'C:\\Users\\colto\\OneDrive\\Desktop\\desktop stuff\\audio_files\ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'ldavis_tuned_'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
8      0.284001 -0.0       1        1  83.851471
6      0.217678  0.0       2        1  15.756539
4     -0.049846  0.0       3        1   0.039205
1     -0.050035 -0.0       4        1   0.039202
5     -0.050150 -0.0       5        1   0.039199
7     -0.050172  0.0       6        1   0.039199
0     -0.050198  0.0       7        1   0.039198
2     -0.050201 -0.0       8        1   0.039198
3     -0.050252 -0.0       9        1   0.039197
10    -0.050268  0.0      10        1   0.039197
11    -0.050264  0.0      11        1   0.039197
9     -0.050293 -0.0      12        1   0.039196, topic_info=             Term          Freq         Total Category  logprob  loglift
143          know  10909.000000  10909.000000  Default  30.0000  30.0000
293         think   3482.000000   3482.000000  Default  29.0000  29.0000
220        people   2968.000000   2968.000000  Default  28.0000  28.0000
43           come   2031.000000   2031.000000  Default  27.0000  27.0000
112            go   6655.000000   6655.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
2614  colonoscopy      0.018961     10.613223  Topic12  -8.4264   1.5169
2370       deeply      0.018952      7.130248  Topic12  -8.4269   1.9141
2932    lunchable      0.018947     10.447911  Topic12  -8.4272   1.5318
108           get      0.018944   3433.588641  Topic12  -8.4273  -4.2633
1877         info      0.018942     16.364900  Topic12  -8.4274   1.0828

[1126 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
888       1  0.749575      able
888       2  0.249036      able
2328      1  0.324840    abroad
2328      2  0.649680    abroad
3344      1  0.464953  absolute
...     ...       ...       ...
325       2  0.126049     would
326       1  0.899302      year
326       2  0.100674      year
4028      1  0.462961    yorker
4028      2  0.462961    yorker

[462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 5, 2, 6, 8, 1, 3, 4, 11, 12, 10])